In [2]:
import pandas as pd
from Bio import SeqIO, AlignIO, Seq
import numpy as np
from collections import Counter
from helper import *
import python_cipres.client as CipresClient

In [3]:
kinetic_file='../output/00_100p_tree/uclust_all_1_kinetic_data.csv'
synth_file='../output/00_100p_tree/synth_data_modified.csv'
seq_file='../output/02_90p_autotrophic_rubisco_tree/auto_uclust_all_0.9_no_outliers.csv'

In [4]:
kinetic_data = pd.read_csv(kinetic_file,names=['kinetic_ID'])
synth_data = pd.read_csv(synth_file,names=['syn_ID'])

if type(seq_file) == str:
    uclust_data = pd.read_csv(seq_file)
elif type(seq_file) == pd.core.frame.DataFrame:
    uclust_data = seq_file.copy()
else:
     raise ValueError('Wrong seq_file format')

uclust_data = uclust_data.merge(kinetic_data, left_on='Query', right_on='kinetic_ID',how='left')
uclust_data = uclust_data.merge(synth_data, left_on='Query', right_on='syn_ID',how='left')

uclust_data['kinetic_flag'] = '-1'
uclust_data['syn_flag'] = '-1'

kinetic_centroid = uclust_data.loc[~pd.isna(uclust_data['kinetic_ID']),'Target'].unique()
syn_centroid = uclust_data.loc[~pd.isna(uclust_data['syn_ID']),'Target'].unique()

uclust_data.loc[uclust_data['Target'].isin(kinetic_centroid),'kinetic_flag'] = '1'
uclust_data.loc[uclust_data['Target'].isin(syn_centroid),'syn_flag'] = '1'


In [9]:
kin = uclust_data[uclust_data.kinetic_flag=='1']
lines = kin['Target'].apply(lambda x: x.split(' ')[0]).values + ',1.0\n'
lines

array(['RBC2_44,1.0\n', 'RBC2_44,1.0\n', 'RBC2_44,1.0\n', ...,
       'gi|357471525|ref|XP_003606047.1|,1.0\n',
       'gi|387767798|gb|AFJ96478.1|,1.0\n', 'RBC2_50,1.0\n'], dtype=object)

In [10]:
lines = uclust_data['Target'].apply(lambda x: x.split(' ')[0]).values + ','+ uclust_data['kinetic_flag'].values+','+uclust_data['syn_flag'].values+'\n'
unique_lines = np.unique(lines)

        
with open('dataset_simplebar_template.txt','r') as file:
    with open('fig2a_ERC_kin.txt', "w") as f1:
        for row in file:
            f1.write(row)
        for line in lines:
            f1.write(line)
        file.close()
        f1.close()

FileNotFoundError: [Errno 2] No such file or directory: 'dataset_simplebar_template.txt'